# Experiment with extracting shape info from confocal images of fish embryos

In [ ]:
from aicsimageio import AICSImage

image_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphMap/data/yx1_samples/20230322/RT/1A_LM010_RT_kikume.nd2"

imObject = AICSImage(image_path)

# Step1: obtain rough "surface prior"
For each xy coordinate in image, determine whether it contains "inside" pixels. Then, find the surface pixel for those that do have one ore more inside pixels.

The most naive approach I can imagine is taking the brightest pixel in each Z column as my surface point. Let's try that first

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# get resolution
res_raw = imObject.physical_pixel_sizes
res_array = np.asarray(res_raw)
res_array = np.insert(res_array, 0, 1)
pixel_size_z = res_array[1]
pixel_size_x = res_array[2]
pixel_size_y = res_array[3]

# find brightest pixel
imData = np.squeeze(imObject.data)
max_pos_z = np.argmax(imData, axis=0)
max_brightness_z = np.max(imData, axis=0)

# generate x and y axes
xg, yg = np.meshgrid(range(max_pos_z.shape[1]), range(max_pos_z.shape[0]))

im95 = np.percentile(max_brightness_z, 90)
x_plot = xg[np.where(max_brightness_z>=im95)]
y_plot = yg[np.where(max_brightness_z>=im95)]
z_plot = max_pos_z[np.where(max_brightness_z>=im95)]

max_pos_z[np.where(max_brightness_z<=im95)] = 0

fig = px.scatter_3d(x=x_plot*pixel_size_x, 
                    y=y_plot*pixel_size_y,
                    z=z_plot*pixel_size_z,
                    opacity=0.002,
                    color=z_plot)
fig.show()

**Try to convert this to a 3D mesh.**

In [ ]:
import pyvista as pv
import pymeshfix as mf
from pymeshfix import MeshFix
from pymeshfix._meshfix import PyTMesh

# await pv.set_jupyter_backend('trame')
np.random.seed(124)
n_samples = x_plot.size
index_vec = range(0, x_plot.size)
mesh_indices = index_vec# np.random.choice(index_vec, n_samples)

# convert xyz coordinates to a point cloud object
xyz_array = np.concatenate((np.reshape(x_plot[mesh_indices]*pixel_size_x, (n_samples, 1)),
                            np.reshape(y_plot[mesh_indices]*pixel_size_y, (n_samples, 1)),
                            np.reshape(z_plot[mesh_indices]*pixel_size_z, (n_samples, 1))), axis=1)


point_cloud = pv.PolyData(xyz_array)

# plot point cloud
point_cloud.plot(jupyter_backend='panel', scalars=z_plot, render_points_as_spheres=True, point_size=1)

**Step 1:** Remove outlier points

In [ ]:
cl, ind = point_cloud.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)

In [ ]:
alpha=50
surf = point_cloud.delaunay_2d(alpha=alpha)
surf.plot(show_edges=False, jupyter_backend='panel')

In [ ]:
# First, fix the holes

# load and fix holes
sf = PyTMesh(False)
sf.load_file('test.ply')
fix_size = 200
sf.fill_small_boundaries(nbe=fix_size, refine=True)

# convert to mesh
vert, faces = sf.return_arrays()
triangles = np.empty((faces.shape[0], 4), dtype=faces.dtype)
triangles[:, -3:] = faces
triangles[:, 0] = 3

surf_fix = pv.PolyData(vert, triangles)

surf_fix.plot(show_edges=False, jupyter_backend='panel')

In [ ]:
surf_smooth = surf_fix.smooth()
surf_smooth.plot(show_edges=False, jupyter_backend='panel')